In [1]:
import numpy as np

def reweight_distribution(original_distribution, temperature=0.5):    
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)    

In [2]:
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

606208/600901 [==============================] - 0s 0us/step
Corpus length: 600893


In [3]:
maxlen = 60                                                            
step = 3                                                               

sentences = []                                                         

next_chars = []                                                        

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('Number of sequences:', len(sentences))

chars = sorted(list(set(text)))                                        
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)       

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)      
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)              
for i, sentence in enumerate(sentences):                               
    for t, char in enumerate(sentence):                                
        x[i, t, char_indices[char]] = 1                                
    y[i, char_indices[next_chars[i]]] = 1  

Number of sequences: 200278
Unique characters: 57
Vectorization...


In [4]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [5]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [6]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [7]:
import random
import sys

for epoch in range(1, 60):                                          
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)                       
    start_index = random.randint(0, len(text) - maxlen - 1)         
    generated_text = text[start_index: start_index + maxlen]        
    print('--- Generating with seed: "' + generated_text + '"')     
    for temperature in [0.2, 0.5, 1.0, 1.2]:                        
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        for i in range(400):                                        
            sampled = np.zeros((1, maxlen, len(chars)))             
            for t, char in enumerate(generated_text):               
                sampled[0, t, char_indices[char]] = 1.              

            preds = model.predict(sampled, verbose=0)[0]            
            next_index = sample(preds, temperature)                 
            next_char = chars[next_index]                           

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)


epoch 1
1565/1565 [==============================] - 110s 70ms/step - loss: 1.9563
--- Generating with seed: "l obtuseness in regard
to it, help human nature forward, whe"
------ temperature: 0.2
l obtuseness in regard
to it, help human nature forward, when the precession of the strenger and the reason and the probound and that is the earred and the the the present of the compressing the commanded and process of the more and the compressing of the proposed and and the eares and suppisence of the precession of the compressions and the science of the science of the preceptions of the exprent and more of the science of the present and proposed and str------ temperature: 0.5
 and more of the science of the present and proposed and strengly that the science of morality propection and the contruted and love attance of the compressions, what as also as to the reases as the philosophy of still in the of the it all morely and or in the howe, it is ary all explession of the trage and that the , 

KeyboardInterrupt: 